In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

from newscrawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [2]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (26.88 seconds)
we have total 414563 articles (354415 downloaded, 60148 trashed)


# 1. URL을 모은다

In [3]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (130.94 seconds): Collecting URLs... yahoo               

100.00% (0.12 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25869,25802,1330
reuters,1080,1080,173
investing.com,940,940,107
euronews,967,967,101
dailymail,2751,2751,90
...,...,...,...
nextbigfuture,35,35,0
thinkprogress,82,82,0
realclearpolitics,307,284,0
heritage,287,287,0


,url,pubs,actual_pub
0,https://cnn.com/2019/10/30/business/grocery-de...,"cnn, axios",cnn
1,https://politico.com/magazine/story/2019/11/05...,"politico, nationalreview",politico
2,https://politico.com/magazine/story/2019/11/05...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/30...,"politico, realclearpolitics",politico
4,https://politico.com/news/2019/11/06/joe-biden...,"politico, realclearpolitics",politico
...,...,...,...
62,https://nbcnews.com/tech/security/saudis-recru...,"msnbc, nbcnews",nbcnews
63,https://nbcnews.com/tech/security/how-fake-tow...,"msnbc, nbcnews",nbcnews
64,https://msnbc.com/mtp-daily/watch/trump-electi...,"msnbc, nbcnews",msnbc
65,https://nbcnews.com/politics/donald-trump/trum...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [4]:
crawl_result = crawler.crawl(); crawl_result

50.00% (76.87 seconds): Crawling... japan-news          

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (625.36 seconds): Crawling... yahoo               



collect                                crawl              
                  collected collected_unique selected downloaded trashed total
all                   25869            25802     1330        773     557  1330
reuters                1080             1080      173         79      94   173
investing.com           940              940      107         42      65   107
euronews                967              967      101          6      95   101
dailymail              2751             2751       90         72      18    90
...                     ...              ...      ...        ...     ...   ...
nextbigfuture            35               35        0          0       0     0
thinkprogress            82               82        0          0       0     0
realclearpolitics       307              284        0          0       0     0
heritage                287              287        0          0       0     0
townhall                314              313        0          0       0     0

[74 rows x 6 columns]

# 3. 뉴스를 저장한다

In [7]:
crawler.record()

Recording to bigquery storage... done (17.24 seconds)


# 4. 결과보고

In [8]:
crawler.report(using='sendgrid')

202


In [7]:
481057/60000

8.017616666666667

In [9]:
60015+133

60148

In [21]:
import requests, json

In [24]:
json.loads(requests.get('https://us-central1-global-news-crawl.cloudfunctions.net/test').text)

{'a': 1, 'b': 2, 'c': 3}

In [13]:
from flask import jsonify

In [19]:
data = {'a':1, 'b':2}
jsonify(a=1, b=3)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [17]:
data

{'a': 1, 'b': 2}